In [8]:
import pandas as pd
import numpy as np
import glob
import os
from io import StringIO
from scipy.stats import wilcoxon

def parse_evaluation_file(file_content):
    lines = file_content.splitlines()
    table_lines = []
    # we don't care about the overall metrics as we analyze per-query differences
    for line in lines:
        if "==== Overall Evaluation Metrics" in line:
            break
        table_lines.append(line)
    table_str = "\n".join(table_lines)
    try:
        df = pd.read_csv(StringIO(table_str), sep='\s+')
    except pd.errors.EmptyDataError:
        raise ValueError("No table data found in the file content.")
    return df

results_folder = "../results"
file_pattern = os.path.join(results_folder, "*.txt")
file_paths = glob.glob(file_pattern)

baseline_keywords = ["bm25", "splade", "colbert", "dpr"]
hybrid_markers = ["fusion", "sequential", "union"]

records = []
for file_path in file_paths:
    with open(file_path, 'r') as f:
        content = f.read()

    df = parse_evaluation_file(content)

    # values are offset in the file, so we have Average precision in the recall column
    # we have recall in the precision column
    values = df['Precision'].astype(float).values

    fname = os.path.basename(file_path).lower()
    is_hybrid = any(marker in fname for marker in hybrid_markers)

    if not is_hybrid:
        matched = [kw for kw in baseline_keywords if kw in fname]
        if matched:
            group = matched[0]
        else:
            group = "other"
        baselines_included = matched
    else:
        group = "hybrid"
        baselines_included = [kw for kw in baseline_keywords if kw in fname]

    records.append({
        "file": os.path.basename(file_path),
        "is_hybrid": is_hybrid,
        "group": group,
        "baselines_included": baselines_included,
        "values": values
    })

df_files = pd.DataFrame(records)

results = []
for baseline in baseline_keywords:
    baseline_subset = df_files[(df_files['is_hybrid'] == False) & (df_files['group'] == baseline)]

    baseline_arrays = list(baseline_subset['values'])
    baseline_avg = np.mean(np.vstack(baseline_arrays), axis=0)

    hybrid_subset = df_files[(df_files['is_hybrid'] == True) &
                             (df_files['baselines_included'].apply(lambda lst: baseline in lst))]

    hybrid_arrays = list(hybrid_subset['values'])
    hybrid_avg = np.mean(np.vstack(hybrid_arrays), axis=0)

    if len(baseline_avg) != len(hybrid_avg):
        print(f"Skipping baseline {baseline} due to mismatched number of queries.")
        continue

    stat, p_val = wilcoxon(baseline_avg, hybrid_avg)

    results.append({
        "Baseline": baseline,
        "Num Baseline Files": len(baseline_subset),
        "Num Hybrid Files": len(hybrid_subset),
        "Baseline Mean": np.mean(baseline_avg),
        "Hybrid Mean": np.mean(hybrid_avg),
        "Wilcoxon Statistic": stat,
        "p-value": p_val,
        "Significant": "Yes" if p_val < 0.05 else "No"
    })

df_wilcoxon = pd.DataFrame(results)
pd.options.display.float_format = '{:.20f}'.format
print("\nWilcoxon Signed-Rank Test Comparison Table:")
print(df_wilcoxon)

example_baseline = baseline_keywords[0]
baseline_subset = df_files[(df_files['is_hybrid'] == False) & (df_files['group'] == example_baseline)]
hybrid_subset = df_files[(df_files['is_hybrid'] == True) &
                         (df_files['baselines_included'].apply(lambda lst: example_baseline in lst))]




Wilcoxon Signed-Rank Test Comparison Table:
  Baseline  Num Baseline Files  Num Hybrid Files  \
0     bm25                   1                28   
1   splade                   1                28   
2  colbert                   1                26   
3      dpr                   1                27   

   Baseline Mean (avg over queries)  Hybrid Mean (avg over queries)  \
0            0.14771052631578945880          0.15379652255639097991   
1            0.12627631578947368096          0.14532894736842108530   
2            0.20751315789473681606          0.16686538461538463762   
3            0.08711842105263158109          0.13946003898635478402   

         Wilcoxon Statistic                p-value Significant  
0 1085.00000000000000000000 0.14439973032038988521          No  
1  806.00000000000000000000 0.00275870496808660074         Yes  
2  576.00000000000000000000 0.00002063957708960586         Yes  
3  187.00000000000000000000 0.00000000015906365703         Yes  


<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
C:\Users\vlad1\AppData\Local\Temp\ipykernel_14568\2729467385.py:19: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(StringIO(table_str), sep='\s+')
